In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from source.sentiment_processor import SentimentProcessor
from source.xml_processor import XMLProcessor
from source.translation_processor import TranslationProcessor
from source.dataset_processor import DatasetProcessor
from tqdm import tqdm

import pandas as pd

In [ ]:
s = SentimentProcessor(no_aspect=False)
xml_processor = XMLProcessor(path='data/apa_dataset/negative.xml')

for x in ['negative', 'positive', 'neutral', 'ambivalent']:
    df = pd.read_csv(f'data/processed_dataset/{x}/articles.csv', header=None)
    xml_processor = XMLProcessor(path=f'data/apa_dataset/{x}.xml')
    # get values from 1st column
    ids = df.iloc[:, 0].values
    documents = xml_processor.process_xml()
    sentiments = xml_processor.get_sentiments()
    df_sent = s.process_sentiments(sentiments, ids)
    df_sent.to_csv(f'data/processed_dataset/{x}/sentiments_with_aspect.csv', index=False)

In [122]:
xml_processor = XMLProcessor(path='data/apa_dataset/negative.xml')
documents = xml_processor.process_xml()

In [135]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def find_entity_mentions(text, entity_name, threshold=75):
    """
    Find all occurrences of an entity name in a text using fuzzy matching.

    Args:
    text (str): The text to search in.
    entity_name (str): The entity name to search for.
    threshold (int): The minimum matching score to consider (default is 80).

    Returns:
    list of dict: A list of dictionaries with 'startingIndex', 'endingIndex', and 'name' of each occurrence.
    """
    occurrences = set()
    words = text.split()
    for i in range(len(words)):
        for j in range(4):
            phrase = ' '.join(words[i:i + j])
            if fuzz.ratio(phrase, entity_name) >= threshold:
                start_index = text.find(phrase)
                end_index = start_index + len(phrase)
                occurrences.add((start_index, end_index, phrase))

    filtered_occurrences = []
    for occ in occurrences:
        is_contained = False
        for other_occ in occurrences:
            if occ != other_occ and occ[0] >= other_occ[0] and occ[1] <= other_occ[1]:
                is_contained = True
                break
        if not is_contained:
            filtered_occurrences.append(occ)

    return {
        'entity_name': entity_name,
        'mentions': [
            {
                'startingIndex': occ[0],
                'endingIndex': occ[1],
                'name': occ[2]
            }
            for occ in filtered_occurrences
        ]
    }
